# <center>**02_data_merge_dicom_clinical**</center>

### Table of Contents

1. **Notebook Overview**  
   - Objective of the metadata integration pipeline  

2. **Environment Setup**  
   - Import of scientific Python libraries  
   - Global configuration  

3. **Data Loading and Initial Inspection**  
   - Loading DICOM and clinical metadata  
   - Dataset dimensions and column schemas  

4. **Column Selection and Harmonization**  
   - Selection and renaming of relevant variables  
   - Normalization of laterality labels  

5. **Change `lat_rx` Values in Clinical Data**

6. **Change Datatypes**  
   - Enforcement of consistent data types  

7. **Dataset Integration**  
   - Merge strategy and validation  

8. **Resolve Duplicated Laterality Columns After Merge**  
   - Final schema preparation  

9. **Save Final Dataset**

10. **Summary and Conclusions**  
   - Key outcomes and relevance for downstream analysis  


### **1. Notebook Overview**

This notebook implements a structured pipeline to merge DICOM-derived imaging metadata with corresponding clinical information into a single, harmonized dataset. The goal is to obtain a clean, consistent, and analysis-ready table that links radiographic studies with relevant clinical variables through shared identifiers, enabling reliable downstream analysis.

Note on Dataset Selection: For clarity and conciseness, the workflow is demonstrated using a small, complementary dataset. The primary datasets used in the main research are not fully shown step-by-step in this notebook due to their substantial volume and the high computational overhead required to handle the full clinical records.

A central objective of this notebook is to ensure that all imaging and clinical records entering subsequent analytical pipelines share the same metadata assumptions and structure. By enforcing consistent identifiers, normalized categorical values, and explicit data types across datasets, this step reduces sources of inconsistency and facilitates reproducible experimentation. The resulting merged dataset serves as a foundational input for later notebooks focused on feature extraction, hybrid machine learning pipelines, and model development.


### **2. Environment Setup**

In [42]:
# Standard library
import os
import warnings
import hashlib

# Third-party
import numpy as np
import pandas as pd
from IPython.display import display


### **3. Data Loading and Initial Inspection**

In [27]:
# Load paths
path_dicom_data_path = "dicom_data.csv"
form_data_path = "form_data.csv"

df_dicom_path_data = pd.read_csv(path_dicom_data_path)
df_clinical_data = pd.read_csv(form_data_path)

# Print shapes
print("DICOM shape:", df_dicom_path_data.shape)
print("Clinical shape:", df_clinical_data.shape)

# Display column names as DataFrames
display(pd.DataFrame(df_dicom_path_data.columns, columns=["DICOM columns"]))
display(pd.DataFrame(df_clinical_data.columns, columns=["Clinical columns"]))


DICOM shape: (308, 12)
Clinical shape: (308, 16)


,DICOM columns
0,nhc
1,episode_id
2,rx_id
3,lat
4,project
5,manufacturer
6,manufacturer_model
7,station_name
8,voilutfunction
9,birthdate


,Clinical columns
0,Unnamed: 0
1,NHC
2,epi_cod
3,rx_cod
4,proje
5,manufacturer
6,manufacturer_model
7,station_name
8,voilutfunction
9,birthdate


### **4. Column Selection and Harmonization**

In [34]:
# DICOM: select and rename
dicom_column_map = {
    "nhc": "NHC",
    "episode_id": "epi_cod",
    "rx_id": "rx_cod",
    "lat": "lat_rx",
    "project": "proje",
    "manufacturer": "manufacturer",
    "manufacturer_model": "manufacturer_model",
    "station_name": "station_name",
    "voilutfunction": "voilutfunction",
    "birthdate": "birthdate",
    "sex": "sex",
    "ei": "ei",
}

df_dicom_path_data_rename = (
    df_dicom_path_data[list(dicom_column_map.keys())]
    .rename(columns=dicom_column_map)
)

# Clinical: select and rename
clinical_column_map = {
    "epi_cod": "epi_cod",
    "NHC": "NHC",
    "date_rx": "date_rx",
    "lat_rx": "lat_rx",
    "complentary_expl": "complentary_expl",
    "label_CalTend": "label_CalTend",
}

df_clinical_data_rename = (
    df_clinical_data[list(clinical_column_map.keys())]
    .rename(columns=clinical_column_map)
)

# Display 
display(df_dicom_path_data_rename.head(1))
display(df_clinical_data_rename.head(1))


,NHC,epi_cod,rx_cod,lat_rx,proje,manufacturer,manufacturer_model,station_name,voilutfunction,birthdate,sex,ei
0,76567,Episodio_V_00094,Episodio_V_00094_I_AP,I,AP,ATS Srl,PRIMO R,PRIMO R,LINEAR,19650116.0,F,100


,epi_cod,NHC,date_rx,lat_rx,complentary_expl,label_CalTend
0,Episodio_V_00089,3fc8e5f59e318eba42b79a2b29f24cad5e7392e007f83c...,16/12/2024,D,Sí,0


### **5. Change lat_rx values in df_clinical_data**

In [33]:
# Normalize laterality labels
df_clinical_data_rename.loc[df_clinical_data_rename["lat_rx"] == "Derecha", "lat_rx"] = "D"
df_clinical_data_rename.loc[df_clinical_data_rename["lat_rx"] == "Izquierda", "lat_rx"] = "I"

# Display first row only
display(df_clinical_data_rename.head(1))


,epi_cod,NHC,date_rx,lat_rx,complentary_expl,label_CalTend
0,Episodio_V_00089,3fc8e5f59e318eba42b79a2b29f24cad5e7392e007f83c...,16/12/2024,D,Sí,0


### **6. Change datatypes**

In [39]:
# Show current dtypes (as DataFrames)
display(pd.DataFrame({"DICOM dtype": df_dicom_path_data_rename.dtypes}))
display(pd.DataFrame({"Clinical dtype": df_clinical_data_rename.dtypes}))

# Ensure join keys are consistent (NHC may be anonymized already)
df_dicom_path_data_rename["NHC"] = df_dicom_path_data_rename["NHC"].astype("string").str.strip()
df_clinical_data_rename["NHC"] = df_clinical_data_rename["NHC"].astype("string").str.strip()

df_dicom_path_data_rename["epi_cod"] = df_dicom_path_data_rename["epi_cod"].astype("string").str.strip()
df_clinical_data_rename["epi_cod"] = df_clinical_data_rename["epi_cod"].astype("string").str.strip()

# Cast DICOM types (avoid forcing NHC to int if anonymized)
df_dicom_path_data_rename["ei"] = pd.to_numeric(df_dicom_path_data_rename["ei"], errors="coerce").astype("float64")
df_dicom_path_data_rename["rx_cod"] = df_dicom_path_data_rename["rx_cod"].astype("string")

dicom_cat_cols = [
    "lat_rx",
    "proje",
    "manufacturer",
    "manufacturer_model",
    "station_name",
    "voilutfunction",
    "sex",
]
df_dicom_path_data_rename[dicom_cat_cols] = df_dicom_path_data_rename[dicom_cat_cols].astype("category")

# Parse birthdate (YYYYMMDD), keep as datetime
birth_str = df_dicom_path_data_rename["birthdate"].astype("string").str.strip()
df_dicom_path_data_rename["birthdate"] = pd.to_datetime(birth_str, format="%Y%m%d", errors="coerce")

# Cast Clinical types (keep NHC as string because it is anonymized)
df_clinical_data_rename["date_rx"] = df_clinical_data_rename["date_rx"].astype("string")

clinical_cat_cols = ["complentary_expl", "lat_rx", "label_CalTend"]
df_clinical_data_rename[clinical_cat_cols] = df_clinical_data_rename[clinical_cat_cols].astype("category")

# Show updated dtypes (as DataFrames)
display(pd.DataFrame({"DICOM dtype": df_dicom_path_data_rename.dtypes}))
display(pd.DataFrame({"Clinical dtype": df_clinical_data_rename.dtypes}))


,DICOM dtype
NHC,int64
epi_cod,string[python]
rx_cod,string[python]
lat_rx,category
proje,category
manufacturer,category
manufacturer_model,category
station_name,category
voilutfunction,category
birthdate,datetime64[ns]


,Clinical dtype
epi_cod,object
NHC,object
date_rx,object
lat_rx,object
complentary_expl,object
label_CalTend,int64


,DICOM dtype
NHC,string[python]
epi_cod,string[python]
rx_cod,string[python]
lat_rx,category
proje,category
manufacturer,category
manufacturer_model,category
station_name,category
voilutfunction,category
birthdate,datetime64[ns]


,Clinical dtype
epi_cod,string[python]
NHC,string[python]
date_rx,string[python]
lat_rx,category
complentary_expl,category
label_CalTend,category


### **7. Dataset Integration**

In [40]:
# Merge datasets using shared identifiers
df_all_data = df_dicom_path_data_rename.merge(
    df_clinical_data_rename,
    on=["NHC", "epi_cod"],
    how="inner",
)

print("Merged shape:", df_all_data.shape)
display(df_all_data.head(1))


Merged shape: (0, 16)


,NHC,epi_cod,rx_cod,lat_rx_x,proje,manufacturer,manufacturer_model,station_name,voilutfunction,birthdate,sex,ei,date_rx,lat_rx_y,complentary_expl,label_CalTend


### **8. Resolve duplicated laterality columns after merge**

In [41]:
# Resolve duplicated laterality columns after merge
if "lat_rx_x" in df_all_data.columns:
    df_all_data["lat_rx"] = df_all_data["lat_rx_x"]
elif "lat_rx_y" in df_all_data.columns:
    df_all_data["lat_rx"] = df_all_data["lat_rx_y"]

# Drop duplicated columns if present
df_all_data.drop(
    columns=[c for c in ["lat_rx_x", "lat_rx_y"] if c in df_all_data.columns],
    inplace=True,
)

# Inspect final schema
print("Final columns:")
print(df_all_data.columns.tolist())


Final columns:
['NHC', 'epi_cod', 'rx_cod', 'proje', 'manufacturer', 'manufacturer_model', 'station_name', 'voilutfunction', 'birthdate', 'sex', 'ei', 'date_rx', 'complentary_expl', 'label_CalTend', 'lat_rx']


### **9. Save**

In [26]:
# Save final dataset
df_all_data.to_csv("all_data_unique_final.csv", index=False)
df_all_data.to_feather("all_data_unique_final.feather")


### **10. Summary and Conclusions**

In this notebook, a complete pipeline for merging DICOM-derived metadata with clinical data has been presented. Through systematic cleaning, normalization, and type enforcement, the two heterogeneous data sources are successfully integrated into a single coherent dataset.

The process highlights the importance of rigorous preprocessing when working with multimodal medical data. Ensuring consistent identifiers, resolving naming conflicts, and enforcing explicit data types are essential steps to avoid silent errors and to enable reliable downstream analyses.

The final output is a standardized and anonymization-safe dataset, exported in commonly used formats for interoperability. This dataset provides a robust basis for future exploratory analysis, feature engineering, and model development, and the pipeline can be readily adapted to similar clinical–imaging integration tasks.